# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# jupyter nbextension enable --py gmaps

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("city_data.csv")
city_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,55.40,54,40,16.11,CL,1610116747
1,1,sao filipe,14.8961,-24.4956,75.60,77,2,16.26,CV,1610116747
2,2,castro,-24.7911,-50.0119,75.04,81,39,4.72,BR,1610116747
3,3,qaanaaq,77.4840,-69.3632,-3.80,87,100,13.60,GL,1610116747
4,4,port alfred,-33.5906,26.8910,75.99,82,65,1.99,ZA,1610116565
...,...,...,...,...,...,...,...,...,...,...
579,579,klaksvik,62.2266,-6.5890,39.20,93,75,17.27,FO,1610116788
580,580,sao felix do xingu,-6.6447,-51.9950,93.31,45,94,1.43,BR,1610116789
581,581,sakaraha,-22.9000,44.5333,84.16,52,50,13.24,MG,1610116789
582,582,ugoofaaru,5.6667,73.0000,83.25,73,100,3.06,MV,1610116789


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
locations = city_data_df[["Lat", "Lng"]]
fig.add_layer(gmaps.heatmap_layer(locations = locations, weights = city_data_df["Humidity"]))
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Narrow down the DataFrame to find your ideal weather condition. For example:

# A max temperature lower than 80 degrees but higher than 70.

# Wind speed less than 10 mph.

# Zero cloudiness.

# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by 
# your API requests to a reasonable number.

# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [5]:
ideal_weather_df = city_data_df[(city_data_df["Max Temp"]<80) & (city_data_df["Max Temp"]>70) & \
                                (city_data_df["Wind Speed"]<10) & (city_data_df["Cloudiness"]==0) ].dropna()
ideal_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
66,66,pisco,-13.7000,-76.2167,73.40,73,0,6.91,PE,1610116753
117,117,beldanga,23.9333,88.2500,70.03,44,0,6.26,IN,1610116580
153,153,esna,25.2934,32.5540,77.00,33,0,7.74,EG,1610116759
166,166,diu,20.7141,70.9822,74.12,63,0,6.15,IN,1610116760
170,170,neuquen,-38.9516,-68.0591,71.60,43,0,6.91,AR,1610116757
313,313,lazaro cardenas,17.9583,-102.2000,75.70,74,0,5.26,MX,1610116771
371,371,viedma,-40.8135,-62.9967,73.99,32,0,7.00,AR,1610116775
418,418,vallenar,-28.5708,-70.7581,79.59,38,0,8.01,CL,1610116778
434,434,broken hill,-31.9500,141.4333,77.00,31,0,9.22,AU,1610116779
465,465,hurghada,27.2574,33.8129,78.80,36,0,9.22,EG,1610116781


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
hotel_df = ideal_weather_df.reset_index().drop(["Unnamed: 0", "index"],axis=1)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,pisco,-13.7000,-76.2167,73.40,73,0,6.91,PE,1610116753
1,beldanga,23.9333,88.2500,70.03,44,0,6.26,IN,1610116580
2,esna,25.2934,32.5540,77.00,33,0,7.74,EG,1610116759
3,diu,20.7141,70.9822,74.12,63,0,6.15,IN,1610116760
4,neuquen,-38.9516,-68.0591,71.60,43,0,6.91,AR,1610116757
5,lazaro cardenas,17.9583,-102.2000,75.70,74,0,5.26,MX,1610116771
6,viedma,-40.8135,-62.9967,73.99,32,0,7.00,AR,1610116775
7,vallenar,-28.5708,-70.7581,79.59,38,0,8.01,CL,1610116778
8,broken hill,-31.9500,141.4333,77.00,31,0,9.22,AU,1610116779
9,hurghada,27.2574,33.8129,78.80,36,0,9.22,EG,1610116781


In [13]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
66,pisco,PE,-13.7000,-76.2167,
117,beldanga,IN,23.9333,88.2500,
153,esna,EG,25.2934,32.5540,
166,diu,IN,20.7141,70.9822,
170,neuquen,AR,-38.9516,-68.0591,
313,lazaro cardenas,MX,17.9583,-102.2000,
371,viedma,AR,-40.8135,-62.9967,
418,vallenar,CL,-28.5708,-70.7581,
434,broken hill,AU,-31.9500,141.4333,
465,hurghada,EG,27.2574,33.8129,


In [17]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
hotel_name = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_name = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_name

['\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>pisco</dd>\n<dt>Country</dt><dd>PE</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>beldanga</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>esna</dd>\n<dt>Country</dt><dd>EG</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>diu</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>neuquen</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>lazaro cardenas</dd>\n<dt>Country</dt><dd>MX</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>viedma</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>vallenar</dd>\n<dt>Country</dt><dd>CL</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>broken hill</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>hurghada<

In [ ]:
# Add marker layer ontop of heat map


# Display figure
